In [1]:
import itertools
from collections import defaultdict
import random
import numpy as np
import pandas as pd
import math
import time
import statistics
import simpy
import json
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt
import tkinter as tk
from PIL import ImageTk

In [2]:
df = pd.read_csv('/Users/pedrorodriguez/Desktop/Springboard/Capstone Project/4th part/production_line_mean_std.csv')

In [3]:
del df['Unnamed: 0']

# Brainstorming

## How to create the algorithm for the simulations

**Process line idea**

1) The daily goal per shift is to produce 600 units, meaning that the first station has to have an input of a minimum of 50 units per hour to achieve the goal. The first station will have an output of 50 units per hour.

2) The units' processing time will be the average of the cycle time of each station.

3) The unit's waiting time will be the difference between the first and second stations. 

### Process flow

Input --> station1 --> station2 --> station3 --> Output

### Formula

station1 = Cycle_time_mean

station2 = Cycle_time_mean

station3 = Cycle_time_mean



In [4]:
#Capacity hourly
station_no = df['station'].nunique()
shift_hr_day = 12
total_pd_line_capacity_hr = station_no * shift_hr_day

#Total Operator time to use machine
lunch_time = 0.5#hr
break_time = 0.5 #hr
shift_time = 11.5 #hr
total_operator_time = shift_hr_day - lunch_time - break_time
no_operator = station_no #The company have one operator per station, meaning that is the same number of station_no.

#Calculating the Production efficiency line 
time_one_unit = (df['Mean'].max())/60
total_time_produced = (600/11)* time_one_unit
total_time_attended = no_operator *  total_operator_time
efficiency = ((total_time_produced)/total_time_attended)

#Calculating the production line capacity
capacity = ((total_pd_line_capacity_hr * total_operator_time)*efficiency)

print('The production line have', station_no, 'stations and is used by the operator', total_operator_time, 'hours per shift. \nThe total capacity of the production line per shift is', int(capacity), 'running with', round((1 - efficiency)*100, 2),'of efficiency.')

The production line have 18 stations and is used by the operator 11.0 hours per shift. 
The total capacity of the production line per shift is 640 running with 73.02 of efficiency.


In [5]:
#Variables 
Input = 52

#Station mean
station1_mean = df.iloc[0]["Mean"]
station2_mean = df.iloc[1]["Mean"]
station3_mean = df.iloc[2]["Mean"]

#Station standard deviation
station1_std = df.iloc[0]["Std"]
station2_std = df.iloc[1]["Std"]
station3_std = df.iloc[2]["Std"]

#Waiting time between stations
Waiting_time_3_2 = station3_mean - station2_mean

Min1 = int(station1_mean - station1_std)
Max1 = int(station1_mean + station1_std)

Min2 = int(station2_mean - station2_std)
Max2 = int(station2_mean + station2_std)

Min3 = int(station3_mean - station3_std)
Max3 = int(station3_mean + station3_std)

print('Station 1 interval' ,Min1, ',',Max1)
print('Station 2 interval' ,Min2, ',',Max2)
print('Station 3 interval' ,Min3, ',',Max3)

Station 1 interval 16 , 17
Station 2 interval 12 , 13
Station 3 interval 15 , 17


env = simpy.Environment()

In [6]:
wait_times = []

class Production_line(object):
    def __init__(self, env, num_operators, num_operators2, num_operators3):
        self.env = env
        self.station1_op = simpy.Resource(env, num_operators)
        self.station2_op = simpy.Resource(env, num_operators2)
        self.station3_op = simpy.Resource(env, num_operators3)

    def station(self, unit):
        yield self.env.timeout(random.randint(Min1, Max1))

    def station2(self, unit):
        yield self.env.timeout(random.randint(Min2, Max2))  

    def station3(self, unit):
        yield self.env.timeout(random.randint(Min3, Max3))   

In [7]:
def unit_through_process(env, unit, production_line):
    arrival_time = env.now

    with production_line.station1_op.request() as request:
        yield request
        yield env.process(production_line.station(unit))

    with production_line.station2_op.request() as request:
        yield request
        yield env.process(production_line.station2(unit))

    with production_line.station3_op.request() as request:
        yield request
        yield env.process(production_line.station3(unit)) 

    #Units heads into the station
    wait_times.append(env.now - arrival_time)

In [8]:
    def run_production_line(env, num_operators, num_operators2, num_operators3):
        
        production_line= Production_line(env, num_operators, num_operators2, num_operators3)

        for unit in range(3):
            env.process(unit_through_process(env, unit, production_line))

        while True:
            yield env.timeout(0.20)
        
        unit += 1
        env.process(unit_through_process(env, unit, pro))

In [9]:
def get_average_wait_time(wait_times):
    average_wait = statistics.mean(wait_times)

In [10]:
def calculate_wait_time(arrival_times, departure_times):
    average_wait = statistics.mean(wait_times)
    # Pretty print the results
    minutes, frac_minutes = divmod(average_wait, 1)
    seconds = frac_minutes * 60
    return round(minutes), round(seconds)

In [11]:
def get_user_input():
    num_operators = input("Input # of stationer 1 working: ")
    num_operators2 = input("Input # of stationer 2 working: ")
    num_operators3 = input("Input # of stationer 3 working: ")
    params = [num_operators, num_operators2, num_operators3]
    if all(str(i).isdigit() for i in params):  # Check input is valid
        params = [int(x) for x in params]
    else:
        print(
            "Could not parse input. The simulation will use default values:",
            "\n1 stationer 1, 1 stationer 2, 1 stationer 3.",
        )
        params = [1, 1, 1]
    return params

In [12]:
def main():
  # Setup
  num_operators, num_operators2, num_operators3 = get_user_input()

  # Run the simulation
  env = simpy.Environment()
  env.process(run_production_line(env, num_operators, num_operators2, num_operators3))
  env.run(until=90)

  # View the results
  mins, secs = get_average_wait_time(wait_times)
  print(
      "Running simulation...",
      f"\nThe average wait time is {mins} minutes and {secs} seconds.",
  )

In [13]:
if __name__ == '__main__':
    main()

TypeError: cannot unpack non-iterable NoneType object